In [1]:
import pickle
import torch
import tensorflow as tf
from kobert_transformers import get_kobert_model
import numpy as np
import pandas as pd
from kobert_transformers import get_tokenizer
import datetime

In [2]:
tf.debugging.set_log_device_placement(True) 
#코드 실행 전에 추가해주면 연산을 할때 어떤 장치에 할당 되었는지 확인할 수 있음

In [3]:
with open('news_normalized.pickle', 'rb') as file:
    data = pickle.load(file)

In [4]:
data.head()

,id,title,date,body,summary,link,body_morphs,summary_morphs,site
0,1,"이총리 ""폭염, 특별재난 준해…전기요금 제한적 특별배려 검토""",2018.07.31. 오전 11:53,"국무회의 주재···체계적 폭염대책 주문""최저임금 명암···변화 수용하되 진통 최소화...",이낙연 국무총리는 31일 계속되는 폭염으로 전기요금 부담에 대한 우려가 커지고 있는...,https://news.naver.com/main/ranking/read.nhn?r...,"[국무회의, 주재···체계적, 폭염대책, 주문, 최저임금, 명암···변화, 수용, ...","[이낙연, 국무총리, 31일, 계속, 는, 폭염, 전기요금, 부담, 우려, 는, 관...",naver
1,2,"김성태 ""盧 탄핵때도 기무사 대응문건 의혹…즉시 제출하라""",2018.07.31. 오전 10:25,"""대통령, 여름휴가 때 노동자·소상공인 생각하길""김성태 자유한국당 원내대표. = 김...","김성태 자유한국당 원내대표는 31일 ""지난 2004년 노무현 전 대통령 탄핵 당시 ...",https://news.naver.com/main/ranking/read.nhn?r...,"[대통령, 여름휴가, 노동자·소상공, 생각하길""김성태, 자유한국당, 원내대표, =,...","[김성태, 자유한국당, 원내대표, 31, 2004년, 노무현, 대통령, 탄핵, 당시...",naver
2,3,"합참, 北 ICBM 제작 정황 보도에 ""면밀히 추적·감시중""",2018.07.31. 오전 11:29,"노재천 공보실장 ""공식확인 부적절···한미간 공조""워싱턴포스트 ""액화연료 사용 IC...",합동참모본부는 31일 북한이 대륙간탄도미사일을 만들고 있는 정황을 포착했다는 보도와...,https://news.naver.com/main/ranking/read.nhn?r...,"[노재천, 공보실장, 공식확인, 부적절···한미간, 공조""워싱턴포스트, 액화연료, ...","[합동참모본부, 31, 북한, 이, 대륙간탄도미사일, 는, 정황, 을, 포착, 었다...",naver
3,4,"기무사 ""盧 대통령 탄핵 당시 계엄검토 문건 작성 안해""",2018.07.31. 오후 2:43,김성태 의원 주장에 반박···지난 정부에서도 관련 내용 확인 국방부 특별수사단이 계...,국군기무사령부는 31일 노무현 전 대통령 탄핵 당시 계엄령 문건을 작성했다는 김성태...,https://news.naver.com/main/ranking/read.nhn?r...,"[김성태, 의원, 주장, 반박···지난, 정부, 에서도, 관련, 내용, 확인, 국방...","[국군기무사령부, 31일, 노무현, 대통령, 탄핵, 당시, 계엄령, 문건, 작성, ...",naver
4,5,"'종전문제 다룰까' 北, 이례적 회담 제의 이어 전향적 태도 보여",2018.07.31. 오후 12:55,"안익산 대표 "" 흔들어 종전선언? 보도, 그럴 수 있다"" 이해 안익산 북측 수석대표...",북한이 이례적으로 남북 장성급 군사회담을 먼저 제의한데 이어 회담에서 전향적인 태도...,https://news.naver.com/main/ranking/read.nhn?r...,"[안익산, 대표, 흔들, 종전선언, ?, 보도, 이해, 안익산, 북측, 수석대표, ...","[북한, 이례적, 남북, 장성급, 군사회담, 먼저, 제의한데, 이, 이, 회담, 전...",naver


## 메모리가없어...5만개로 줄여보기

In [6]:
data = data[:50000]

#### 본문 정수인코딩 + padding

In [7]:
bmorphs = data.body_morphs

In [13]:
l=[]
m = 512
cnt= 0
for n,_ in enumerate(bmorphs):
    if len(_)>m:
        l.append(n)
        cnt+=1

In [14]:
cnt

28328

In [10]:
tok = get_tokenizer()

In [40]:
bodytok = []
for _ in bmorphs:
    bid = tok.convert_tokens_to_ids(_)
    if(len(bid)<512):
        new = bid+[1]*(512-len(bid)) #padded part = 1
    elif(len(bid)>512):
        new = bid[:512]
    bodytok.append(new)       

In [41]:
print(len(bodytok[1]))
print(bodytok[1])

512
[5821, 0, 0, 0, 0, 0, 254, 0, 0, 0, 147, 0, 0, 5821, 0, 0, 0, 0, 0, 8, 0, 7782, 5585, 0, 0, 6556, 0, 5511, 0, 0, 0, 0, 6903, 0, 0, 7096, 0, 6210, 0, 0, 0, 7157, 7096, 0, 6157, 0, 5760, 0, 6544, 0, 0, 0, 0, 6888, 54, 5585, 0, 5760, 8, 5725, 5821, 0, 0, 0, 5516, 0, 0, 0, 6903, 0, 7088, 0, 7782, 0, 0, 7096, 0, 5439, 0, 8, 0, 0, 6484, 0, 0, 5507, 0, 0, 6903, 0, 7782, 8, 5439, 0, 7782, 6888, 54, 7096, 7096, 6855, 8, 0, 0, 6903, 5512, 7096, 0, 7088, 0, 7782, 5760, 7884, 5760, 5503, 5330, 0, 0, 6903, 0, 0, 8, 7096, 6008, 0, 5330, 0, 7782, 0, 0, 5859, 0, 0, 0, 0, 5886, 0, 0, 7096, 0, 6157, 7096, 5330, 5439, 0, 7096, 5770, 0, 7096, 5770, 7782, 0, 0, 6797, 0, 7782, 0, 5886, 0, 8, 5439, 0, 7782, 6888, 54, 0, 0, 0, 6896, 6855, 7782, 6855, 8, 0, 0, 5585, 0, 0, 7096, 0, 0, 6896, 0, 7782, 5760, 0, 7088, 5439, 6797, 0, 6855, 0, 0, 6116, 0, 8, 0, 5760, 0, 0, 7096, 0, 0, 7088, 5439, 0, 6116, 6940, 0, 6116, 7782, 0, 7096, 0, 0, 5330, 6166, 5760, 0, 7088, 7782, 6855, 0, 6797, 8, 5439, 0, 7782, 6888, 5

In [42]:
print(len(bodytok[2]))
print(bodytok[2])

512
[0, 0, 0, 0, 0, 0, 0, 0, 93, 463, 0, 0, 0, 0, 0, 0, 147, 6415, 0, 0, 0, 0, 0, 0, 0, 7088, 0, 5477, 5875, 7088, 0, 7461, 478, 0, 0, 0, 7088, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 7945, 0, 8, 0, 0, 0, 6415, 0, 7337, 0, 5477, 0, 6896, 7782, 0, 7461, 478, 0, 7141, 8, 7782, 54, 0, 0, 5760, 0, 0, 7096, 6415, 0, 0, 0, 7088, 0, 7782, 6888, 5439, 0, 7782, 6888, 54, 0, 5760, 7460, 0, 7457, 0, 0, 7086, 0, 7088, 0, 0, 0, 0, 7782, 5760, 0, 93, 463, 0, 0, 7096, 6855, 5760, 0, 7096, 7141, 5782, 5439, 0, 7782, 6888, 54, 0, 0, 7782, 0, 0, 0, 0, 0, 7096, 0, 0, 5760, 0, 0, 7782, 0, 7141, 5782, 5439, 6415, 7096, 0, 7782, 0, 6273, 7141, 5760, 7430, 7369, 0, 7096, 0, 0, 5451, 7096, 5782, 54, 0, 5760, 6415, 7096, 7859, 5766, 7088, 0, 7788, 7141, 5782, 5439, 0, 7782, 0, 5760, 5782, 5439, 7782, 6888, 54, 0, 0, 6256, 5821, 7096, 0, 0, 7096, 5782, 5439, 0, 0, 6901, 6415, 7096, 0, 0, 6929, 0, 7782, 0, 7088, 5997, 7782, 5439, 0, 7782, 6888, 54, 0, 7095, 0, 7096, 0, 5760, 0, 6855, 7318, 7318, 0, 7945, 5886, 0, 6415, 70

In [43]:
encoder_input = np.array(bodytok)
encoder_input.shape

(50000, 512)

#### summary 정수인코딩 + padding

In [18]:
smorphs = data.summary_morphs

In [19]:
smorphs = smorphs.apply(lambda x : ['sos']+x+['eos'])

In [20]:
smorphs.head()

0    [sos, 이낙연, 국무총리, 31일, 계속, 는, 폭염, 전기요금, 부담, 우려,...
1    [sos, 김성태, 자유한국당, 원내대표, 31, 2004년, 노무현, 대통령, 탄...
2    [sos, 합동참모본부, 31, 북한, 이, 대륙간탄도미사일, 는, 정황, 을, 포...
3    [sos, 국군기무사령부, 31일, 노무현, 대통령, 탄핵, 당시, 계엄령, 문건,...
4    [sos, 북한, 이례적, 남북, 장성급, 군사회담, 먼저, 제의한데, 이, 이, ...
Name: summary_morphs, dtype: object

In [21]:
l=[]
m = 200
cnt= 0
for n,_ in enumerate(smorphs):
    if len(_)>m:
        l.append(n)
        cnt+=1

In [22]:
cnt

186

In [23]:
l[:5]

[289, 384, 1086, 1194, 1355]

In [24]:
tok = get_tokenizer()

just to check unique words

In [27]:
tmp = []
for _ in smorphs:
    el_id = tok.convert_tokens_to_ids(_)
    tmp.append(el_id)       

In [28]:
hi = set()
for lst in tmp:
    for _ in lst:
        if _ not in hi:
            hi.add(_)

In [29]:
print(hi)

{0, 5, 6, 8, 11, 15, 17, 41, 42, 45, 46, 47, 49, 50, 53, 54, 55, 56, 57, 59, 60, 63, 65, 68, 71, 73, 75, 77, 79, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 93, 97, 101, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 119, 122, 124, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 205, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 253, 254, 257, 258, 259, 264, 266, 267, 268, 269, 270, 271, 274, 275, 276, 277, 278, 279, 280, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 298, 299, 300, 301, 303, 304, 305, 306,

In [32]:
hi2 = set()
for lst in bodytok:
    for _ in lst:
        if _ not in hi2:
            hi2.add(_)

In [33]:
print(hi2)

{0, 1, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 40, 41, 42, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 63, 64, 65, 68, 71, 73, 75, 77, 79, 81, 82, 83, 85, 86, 87, 88, 89, 90, 91, 93, 97, 101, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 119, 122, 124, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 205, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 252, 253, 254, 257, 258, 259, 260, 261, 262, 264, 265, 266, 267, 268, 269, 270, 271, 273, 274, 275, 276, 277, 278, 279, 280, 282, 284, 285, 286, 287, 288, 289

초반에는 sos를 8002, eos를 8003으로 뒀었다. 그러나 추후 임베딩 시 해당 해당 인덱스를 인식못하는 에러가 발생했다 . 따라서 둘 다에 존재하지 않는 2,3에 sos,eos를 둬도 될것같다.

sos=2, eos=3  
unknown = 0  
padded part = 1

In [34]:
summarytok = []
for _ in smorphs:
    el_id = tok.convert_tokens_to_ids(_)
    el_id[0] = 2 #sos
    el_id[-1] = 3 #eos
    if(len(el_id)<200):
        new = el_id+[1]*(200-len(el_id)) #padded part = 1
    elif(len(el_id)>200):
        new = el_id[:200]
        new[-1] = 3 #eos
    summarytok.append(new)       

In [35]:
print(len(summarytok[1]))
print(summarytok[1])

200
[2, 0, 0, 0, 147, 0, 0, 5821, 0, 0, 0, 0, 0, 7782, 8, 5439, 0, 7782, 6888, 5585, 0, 5760, 0, 6556, 0, 5511, 6903, 0, 0, 0, 0, 6903, 0, 0, 7096, 0, 6210, 0, 0, 0, 7157, 7096, 0, 6157, 0, 5760, 0, 6544, 5886, 0, 0, 7088, 0, 0, 6888, 54, 5585, 0, 0, 5760, 8, 5725, 7207, 5821, 0, 0, 0, 5516, 0, 0, 0, 6903, 0, 7088, 0, 7782, 0, 0, 7096, 0, 5330, 5439, 0, 8, 0, 0, 6484, 0, 0, 5859, 5507, 0, 0, 6903, 0, 6855, 7782, 0, 8, 5439, 0, 7782, 6888, 54, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [36]:
print(len(summarytok[289]))
print(summarytok[289])

200
[2, 0, 0, 0, 0, 0, 0, 0, 0, 6634, 7096, 0, 0, 0, 0, 0, 0, 0, 0, 7095, 0, 0, 5477, 0, 0, 15, 0, 0, 0, 7088, 7945, 7782, 0, 6888, 0, 7086, 0, 0, 0, 7096, 5788, 0, 15, 6005, 0, 7095, 0, 0, 6896, 0, 7095, 0, 7096, 0, 0, 0, 5886, 0, 0, 7088, 0, 7096, 7295, 0, 6116, 0, 8, 0, 5886, 0, 0, 6896, 0, 0, 7289, 6896, 0, 0, 0, 7096, 0, 5886, 0, 6896, 0, 7788, 0, 6421, 7157, 0, 0, 6645, 7088, 0, 6855, 0, 7096, 0, 0, 6116, 7945, 7782, 0, 7088, 0, 7788, 8, 5439, 6300, 7782, 6888, 54, 0, 7086, 8, 0, 6366, 0, 0, 0, 7086, 6867, 6896, 5454, 5886, 0, 0, 7096, 0, 15, 0, 6271, 0, 0, 15, 7096, 0, 15, 7903, 0, 5477, 0, 15, 7096, 6889, 0, 0, 5760, 0, 0, 0, 6116, 0, 6896, 7012, 7782, 6855, 15, 6258, 0, 0, 15, 0, 6016, 0, 7078, 0, 6356, 0, 6116, 0, 7782, 6886, 46, 6165, 7096, 0, 5886, 0, 334, 6116, 0, 7782, 6855, 0, 7088, 0, 7782, 6892, 46, 0, 7012, 0, 0, 5886, 0, 0, 7096, 5561, 3]


In [37]:
hi = set()
for lst in summarytok:
    for _ in lst:
        if _ not in hi:
            hi.add(_)

In [38]:
len(hi) #num of words in summary

2045

In [44]:
decoder_input = np.array(summarytok)

In [45]:
decoder_input.shape

(50000, 200)

추후 encoder_input / decoder_input 은 embedding을 통해 3Dtensor로 변환

In [46]:
MAX_LEGNTH = 512 #for padding
NUM_WORDS = 2045 #본문 기사 내 가장 많이 사용된 3000단어?(or 전체 단어수로 할지 미정)
VECTOR_SIZE = 768
MAX_SUMMARIZATION_LENGTH = 200

#### Make decoder output (For dense layer)

In [49]:
mapping = dict()
for n,_ in enumerate(hi):
    mapping[n] = _

In [50]:
print(mapping)

{0: 0, 1: 1, 2: 2, 3: 3, 4: 5, 5: 6, 6: 8, 7: 11, 8: 15, 9: 17, 10: 41, 11: 42, 12: 45, 13: 46, 14: 47, 15: 49, 16: 50, 17: 53, 18: 54, 19: 55, 20: 56, 21: 57, 22: 59, 23: 60, 24: 63, 25: 65, 26: 68, 27: 71, 28: 73, 29: 75, 30: 77, 31: 79, 32: 81, 33: 82, 34: 83, 35: 85, 36: 86, 37: 87, 38: 88, 39: 89, 40: 90, 41: 91, 42: 93, 43: 97, 44: 101, 45: 104, 46: 105, 47: 106, 48: 107, 49: 108, 50: 109, 51: 110, 52: 111, 53: 112, 54: 113, 55: 114, 56: 115, 57: 116, 58: 119, 59: 122, 60: 124, 61: 127, 62: 128, 63: 129, 64: 130, 65: 131, 66: 132, 67: 133, 68: 134, 69: 135, 70: 136, 71: 137, 72: 138, 73: 139, 74: 140, 75: 141, 76: 142, 77: 144, 78: 145, 79: 146, 80: 147, 81: 148, 82: 149, 83: 150, 84: 151, 85: 152, 86: 153, 87: 154, 88: 155, 89: 157, 90: 160, 91: 161, 92: 162, 93: 163, 94: 164, 95: 165, 96: 166, 97: 167, 98: 168, 99: 169, 100: 170, 101: 171, 102: 172, 103: 175, 104: 176, 105: 177, 106: 178, 107: 179, 108: 180, 109: 181, 110: 182, 111: 183, 112: 184, 113: 185, 114: 186, 115: 187, 

In [51]:
list(mapping.keys())[list(mapping.values()).index(3)]

3

In [57]:
decoder_target = np.zeros((50000,200,2045),dtype='float32')

In [58]:
for i, target in enumerate(summarytok):
    for n, word in enumerate(target):
        decoder_target[i,n,list(mapping.keys())[list(mapping.values()).index(word)]]=1

In [59]:
decoder_target[0]

array([[0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

# Train Model

In [60]:
MAX_LEGNTH = 512
NUM_WORDS = 2064 #본문 기사 내 가장 많이 사용된 3000단어?(or 전체 단어수로 할지 미정)
VECTOR_SIZE = 768
MAX_SUMMARIZATION_LENGTH = 200

In [61]:
encoder_input.shape

(50000, 512)

In [62]:
decoder_input.shape

(50000, 200)

In [63]:
decoder_target.shape

(50000, 200, 2045)

In [64]:
pytorch_kobert_model = get_kobert_model()

In [65]:
def kobert_convert(integer_encoding, kobert_model):
  '''정수 인코딩된 자료를 pytorch kobert 모델 통과해서 tensorflow의 tensor로 변환'''
  return tf.convert_to_tensor(kobert_model(torch.LongTensor(integer_encoding))[0].detach())

In [66]:
from keras import Model
from keras.layers import Embedding, LSTM, Dropout, Input, Dense, dot, concatenate
from keras.optimizers import Adam

In [76]:
class Seq2Seq_Attention():
    def __init__(self):
        self.max_length = 512
        self.VECTOR_SIZE = 768
        self.max_summary_length = 200
        self.num_words = 2045
        
        self.sos =  2
        self.eos =  3
        
        self.optimizer = Adam()
        
        encoder_inputs = Input(shape=(self.max_length,self.VECTOR_SIZE))
        decoder_inputs = Input(shape=(self.max_summary_length,self.VECTOR_SIZE))
        
        self.att = self.build_att()

        self.att.compile(loss='categorical_crossentropy', optimizer=self.optimizer)
        print(self.att.summary())
        
        
    def build_att(self):
        def lstm(inputs, hs, seq=True, initial=None):
            output,h,c = LSTM(hs, return_state=True, return_sequences=seq)(inputs, initial_state=initial) 
            #return_seq=False & return_state=True: return only last h, c
            #return_seq=Ture & return_state=True: return all h, c
            return output, h, c
        
        def fc(n_h_layers, inputs, hn):
            for _ in range(n_h_layers):
                d = Dense(hn, activation='tanh')(inputs)
            if n_h_layers==0: 
                d = inputs
            output = Dense(hn, activation='softmax')(d)
            #모든 2064개 단어에 대한 확률값 (해당 위치에서의)
            return output
        
        #(encoder input) already embedded from koBERT(vector size 만큼)
        encoder_inputs = Input(shape=(self.max_length,self.VECTOR_SIZE))
        encoder_outputs, h, c = lstm(encoder_inputs, 256)  #Discard encoder outputs
        print(encoder_outputs)
        init_states = [h,c]
        
        decoder_inputs = Input(shape=(self.max_summary_length,self.VECTOR_SIZE))
        decoder_outputs, _, _ = lstm(decoder_inputs, 256, initial=init_states) #Discard encoder outputs
        
        value = Dense(5000, activation='tanh')(encoder_outputs)
        query = Dense(5000, activation='tanh')(decoder_outputs)
        print(value, query) #300x5000 두개
        
        attention = dot([query, value],axes=[2,2])
        print(attention) #300x300
        
        attention_softmaxed = fc(0, attention, self.max_length)
        print(attention_softmaxed) #300x300
        print(encoder_outputs)
        
        weighted = dot([attention_softmaxed, encoder_outputs], axes=[2,1]) #give weights to encoder outputs(=각 토큰)
        print(attention_softmaxed)
        
        print(weighted)
        
        decoder_for_final = concatenate([weighted, decoder_outputs]) #weighted token(2064x256) + decoder output(2064x256)
        #or add? 둘중에 성능좋은거 고르기
        
        decoder_final = fc(1, decoder_for_final, self.num_words)
        print(decoder_final)
        
        mod = Model([encoder_inputs, decoder_inputs], decoder_final) 
        return mod
    
    #Get data of batch size
    def load_batch(self, batch_size=32):
        self.n_batches = int(55000/32)
        for i in range(self.n_batches):
            batch_e = encoder_input[i*batch_size:(i+1)*batch_size]
            batch_d = decoder_input[i*batch_size:(i+1)*batch_size]
            batch_d_o = decoder_target[i*batch_size:(i+1)*batch_size]

            enc_ins, dec_ins, dec_out = [],[],[]
            for enc_in, dec_in, dec_o in zip(batch_e, batch_d, batch_d_o):
                enc_ins.append(enc_in)
                dec_ins.append(dec_in)
                dec_out.append(dec_o)

            yield enc_ins, dec_ins, dec_out
    
    #train model
    def train(self, epochs, batch_size = 32):
        start_time = datetime.datetime.now()
        
        for epoch in range(epochs):
            for batch_i, (enc_batch, dec_batch, dec_out_batch) in enumerate(self.load_batch(batch_size)):
                
                enc_batch = kobert_convert(enc_batch, pytorch_kobert_model)
                dec_batch = kobert_convert(dec_batch, pytorch_kobert_model)
                
                if batch_i%100==0:
                    print(np.array(enc_batch).shape, np.array(dec_batch).shape, np.array(dec_out_batch).shape)
                
                enc_batch = np.array(enc_batch)
                dec_batch = np.array(dec_batch)
                dec_out_batch = np.array(dec_out_batch)
                
                att_loss = self.att.train_on_batch([enc_batch, dec_batch], dec_out_batch)
                
                elapsed_time = datetime.datetime.now() - start_time
                
                if batch_i%100==0:
                    print("[Epoch %d/%d] [Batch %d/%d] Loss: %05f Time:%s"%(epoch, epochs, batch_i, self.n_batches, att_loss, elapsed_time))

In [77]:
model = Seq2Seq_Attention()

Tensor("lstm_2/Identity:0", shape=(None, 512, 256), dtype=float32)
Tensor("dense_5/Identity:0", shape=(None, 512, 5000), dtype=float32) Tensor("dense_6/Identity:0", shape=(None, 200, 5000), dtype=float32)
Tensor("dot_2/Identity:0", shape=(None, 200, 512), dtype=float32)
Tensor("dense_7/Identity:0", shape=(None, 200, 512), dtype=float32)
Tensor("lstm_2/Identity:0", shape=(None, 512, 256), dtype=float32)
Tensor("dense_7/Identity:0", shape=(None, 200, 512), dtype=float32)
Tensor("dot_3/Identity:0", shape=(None, 200, 256), dtype=float32)
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Tensor("dense_9/Identity:0", shape=(None, 200, 2045), dtype=float32)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Out

In [ ]:
model.train(epochs=10, batch_size=64)

(64, 512, 768) (64, 200, 768) (64, 200, 2045)
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op OptimizeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MakeIterator in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/repl